This script generates Land-based yearly water masks for a period from 1987 to 2019, using the water index proposed by `(Fisher et al., 2016)` and a global threshold.

## User-defined Input

In [ ]:
# Path to parcels
path_parcel = '/content/drive/MyDrive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Coastline_parcels_polygon/parcels.geojson'

In [ ]:
# IDs of the parcels to export
#parcelIDs = [693,694,696,697,698,699,700,701,702]
parcelIDs = [661] # patch parcel 661, malaysia

In [ ]:
# Output Folder (on Drive)
drive_folder = "wmsk_wiFi"

## Load Packages

In [ ]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

# Connect to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

Mounted at /content/drive
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=k9dObUXbAdCSSQW5wyuA1CHIvdMfWeilD5NGQCBCdC4&tc=8GukiB26QzoRyfqBG0RhRKwmELpZ7Rn7c1PfzUfwPz4&cc=C_MtA6zcQJuDgS8wScWwkFNQLQKM0fnEJXIgzbY2IYM

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjb7zAaPPt02Spv-wWrc24w6bY1oHmta0LsfwoKjRQj1pvIShpvk6o

Successfully saved authorization token.


In [ ]:
!pip install geojson

import os
from glob import glob
import geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Level-0 Functions

In [ ]:
# Function to Mask Clouds
def maskCloud(image):
  cloudMask = image.select(['pixel_qa']).bitwiseAnd(1<<5).eq(0)
  cloudShadowMask = image.select(['pixel_qa']).bitwiseAnd(1<<3).eq(0)
  final_mask = cloudMask.And(cloudShadowMask)
  return image.updateMask(final_mask)

In [ ]:
# Function to add water Index bands
def addIndex(image):

  wiFi = image.expression("1.7204 + 171*green +3*red - 70*nir - 45*swir1 - 71*swir2",{
    'green': image.select(['green']),
    'red': image.select(['red']),
    'nir': image.select(['nir']),
    'swir1': image.select(['swir1']),
    'swir2': image.select(['swir2'])
  }).rename(['wiFi'])
  # https://www.sciencedirect.com/science/article/abs/pii/S0034425715302753
  
  return image.addBands([wiFi])

In [ ]:
# Function of Otsu's Method
def thresholding(image):
  
  waterIndex = 'wiFi'
  thrhs = -133235.6
  # Based on Accuracy Assessment

  watermask = image.select([waterIndex]).gt(thrhs).rename(['wmsk_'+waterIndex]).unmask(2) 
  # Water = 1, Non-water = 0, No-Data = 2

  return image.addBands(watermask)\
              .select(['wmsk_'+waterIndex]) \
              .copyProperties(image) \
              .set('system:time_start', ee.Date.fromYMD(image.get('year'), 12, 31)) \

## Level-1 Function

In [ ]:
def create_watermsk(ROI):
  roi = ee.Geometry(ROI)

  # Load Landsat Archives
  ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
          .filterDate('2013', '2020') \
          .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
          .filterBounds(roi) \
          .sort("system:time_start", True)
  ls7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
        .filterDate('1999', '2020') \
        .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
        .filterBounds(roi) \
        .sort("system:time_start", True)
  ls5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
        .filterDate('1984', '2013') \
        .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
        .filterBounds(roi) \
        .sort("system:time_start", True)
  ls4 = ee.ImageCollection("LANDSAT/LT04/C01/T1_SR") \
        .filterDate('1984', '1994') \
        .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
        .filterBounds(roi)\
        .sort("system:time_start", True)

  # Merge Collections + Mask Cloud + Add water index bands
  ls_all = ls8.merge(ls7).merge(ls5).merge(ls4)
  ls_all = ls_all.sort('system:time_start', True).map(maskCloud).map(addIndex)

  # Temporally reduced to individual one-year stack
  years = ee.List.sequence(1984, 2019)
  ls_Treduced = ee.ImageCollection.fromImages(
      years.map(lambda YEAR: ls_all.filter(ee.Filter.calendarRange(YEAR, YEAR, 'year')) \
                            .median() \
                            .set({'year': YEAR, 'system:time_start': ee.Date.fromYMD(YEAR, 12, 31)})))
  
  # Filter out years where no data are available
  # Add "bandlength" as new property
  ls_filtered = ls_Treduced.map(lambda image: image.set('bandlength', image.bandNames().size()))
  # Filter out the years where no data are available
  ls_filtered = ls_filtered.filterMetadata('bandlength', 'not_equals', 0)

  # Thresholding, Create water mask
  ls_wmsk = ls_filtered.map(thresholding)

  img_wmsk = ls_wmsk.toBands()
  return img_wmsk

## Export

In [ ]:
# Generator Function for exporting
def genFun(n): # n = len(parcelIDs)

  with open(path_parcel) as f:
    parcels_geojson = geojson.load(f)

  samples = [i for i in parcels_geojson['features'] if i['properties']['TARGET_FID'] in parcelIDs]
  rois = [i['geometry'] for i in samples]
  roi_ids = [i['properties']['TARGET_FID'] for i in samples]
  
  i = 0
  while i < n:
    image = create_watermsk(rois[i])
    task_config = {
        'description': 'p-'+str(roi_ids[i]),
        'fileNamePrefix': 'p-'+str(roi_ids[i]),
        'crs': 'EPSG:4326',
        'scale': 30,  
        'region': ee.Geometry(rois[i]).getInfo()['coordinates'],
        'folder': drive_folder,
        'skipEmptyTiles': True,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 10e12
    }
    task = ee.batch.Export.image.toDrive(image, **task_config)
    yield task.start()
    i += 1

gen = genFun(len(parcelIDs))

In [ ]:
# Exhaust Generator
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)